Code importing fashion picture dataset and tuning a neural network to classify as one of ten options.

In [1]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

1.14.0


In [2]:
import matplotlib.pyplot as plt
plt.imshow(training_images[0])
print(training_labels[0])
print(training_images[0])

9
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   1   0   0  13  73   0
    0   1   4   0   0   0   0   1   1   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3   0  36 136 127  62
   54   0   0   0   1   3   4   0   0   3]
 [  0   0   0   0   0   0   0   0   0   0   0   0   6   0 102 204 176 134
  144 123  23   0   0   0   0  12  10   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0 155 236 207 178
  107 156 161 109  64  23  77 130  72  15]
 [  0   0   0   0   0   0   0   0   0   0   0   1   0  69 207 223 218 216
  216 163 127 121 122 146 141  88 172  66]
 [  0   0   0   0   0   0   0   0   0   1   1   1   0 200 232 

In [3]:
training_images = training_images / 255.0
test_images = test_images / 255.0

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

W0706 20:58:13.143510 30912 deprecation.py:506] From C:\Users\Abhijit\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
model.compile(optimizer=tf.train.AdamOptimizer(),
             loss = 'sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 5s 80us/sample - loss: 0.4996 - acc: 0.8242
Epoch 2/5
60000/60000 [==============================] - 5s 83us/sample - loss: 0.3714 - acc: 0.8652
Epoch 3/5
60000/60000 [==============================] - 5s 80us/sample - loss: 0.3309 - acc: 0.8803
Epoch 4/5
60000/60000 [==============================] - 5s 82us/sample - loss: 0.3074 - acc: 0.8868
Epoch 5/5
60000/60000 [==============================] - 5s 80us/sample - loss: 0.2902 - acc: 0.8923


In [6]:
model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 0s 39us/sample - loss: 0.3396 - acc: 0.8768


[0.3395915552854538, 0.8768]

Viewing a single image classification and most likely prediction

In [7]:
classifications = model.predict(test_images)
print(classifications[0])
print(test_labels[0])

[5.7976307e-05 1.1885827e-06 3.7777602e-07 6.2279634e-07 1.3863173e-05
 3.2943361e-02 1.6630889e-05 1.8093299e-02 7.8862518e-05 9.4879383e-01]
9


Using more neurons in the second layer to improve performance

In [8]:
model2 = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(1024, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

model2.fit(training_images, training_labels, epochs=5)
model2.evaluate(test_images, test_labels)

classifications = model2.predict(test_images)

print(classifications[0])
print(test_labels[0])

Epoch 1/5
60000/60000 [==============================] - 21s 343us/sample - loss: 0.4724
Epoch 2/5
60000/60000 [==============================] - 20s 325us/sample - loss: 0.3568
Epoch 3/5
60000/60000 [==============================] - 20s 335us/sample - loss: 0.3205
Epoch 4/5
60000/60000 [==============================] - 20s 336us/sample - loss: 0.2957
Epoch 5/5
10000/10000 [==============================] - 1s 143us/sample - loss: 0.3620
[2.1863372e-05 6.5840289e-10 1.8270100e-08 1.0564645e-09 1.5056614e-07
 9.6502487e-04 1.9978908e-07 1.4841241e-01 3.0620889e-08 8.5060030e-01]
9


Using callbacks to end after just enough training

In [10]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('loss')<0.30):
      print("\nReached 70% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

model3 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model3.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model3.fit(training_images, training_labels, epochs=5, callbacks=[callbacks])

Epoch 1/5
60000/60000 [==============================] - 10s 159us/sample - loss: 0.4760
Epoch 2/5
60000/60000 [==============================] - 11s 181us/sample - loss: 0.3616
Epoch 3/5
60000/60000 [==============================] - 11s 185us/sample - loss: 0.3227
Epoch 4/5
59616/60000 [============================>.] - ETA: 0s - loss: 0.2978
Reached 70% accuracy so cancelling training!
60000/60000 [==============================] - 9s 151us/sample - loss: 0.2977
